we can use this paper with we want train for only one speleothem
https://arxiv.org/pdf/1605.06065.pdf

In [38]:
import pandas as pd
from torch import nn
import torch
from torch.utils.data import Dataset, random_split, DataLoader
from sklearn.preprocessing import StandardScaler

# Data

In [41]:
class SpeleothemDating(Dataset):
    def __init__(self, annotations_file, normalize=True):
        self.speleothem = pd.read_csv(annotations_file)
        
        if normalize:
            self.normalize()

        self.x_label = ["depth_dating", "latitude", "longitude", "elevation", "entity_id_site"]
        self.y_label = ["corr_age"]
        
        x = self.speleothem.loc[:, self.x_label].values
        y = self.speleothem.loc[:, self.y_label].values
        
        self.x_train = torch.tensor(x, dtype=torch.float32)
        self.y_train = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.speleothem)

    def __getitem__(self, idx):
        return self.x_train[idx], self.y_train[idx]
    
    def normalize(self):
        std_scaler = StandardScaler()
        self.speleothem = pd.DataFrame(std_scaler.fit_transform(self.speleothem), columns=self.speleothem.columns)


In [42]:
speleothem = SpeleothemDating("training_data.csv")

In [43]:
train_size = int(0.8 * len(speleothem))
test_size = len(speleothem) - train_size
train_dataset, test_dataset = random_split(speleothem, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [44]:
train_features, train_labels = next(iter(train_dataloader))
train_features, train_labels

(tensor([[-0.3331,  0.8161, -1.5272, -0.5049,  0.5153],
         [-0.2709,  0.3119,  0.9551, -0.4150,  0.3410],
         [ 2.0435,  0.3352,  0.9514, -0.3658,  0.6083],
         [-0.6485,  0.6338,  1.0029,  0.6811,  1.1430],
         [-0.6407,  0.1898,  0.7341,  0.8076, -0.9492],
         [-0.4869,  0.1898,  0.7341,  0.8076, -0.9492],
         [ 0.0481,  0.6243, -1.8175,  2.3675, -1.1351],
         [ 0.0889, -1.9935,  1.7569, -0.5485,  0.2131],
         [ 0.2395,  0.9455, -0.2804,  1.6508, -0.2402],
         [-0.6551, -0.8740, -1.3540,  1.7491,  0.7129],
         [-0.4444,  0.6155, -0.4060, -0.8998,  1.3987],
         [-0.3372,  0.7690, -1.9138,  0.8217, -0.0774],
         [ 3.8679, -0.5396,  1.0162, -0.7944, -0.3448],
         [-0.1412,  0.4562,  0.9120,  0.3158,  0.9105],
         [ 1.5395,  0.8148, -0.4410, -0.8857,  1.6312],
         [-0.0132,  0.1154,  0.3095,  0.1190,  0.0737]]),
 tensor([[-0.6508],
         [-0.6038],
         [-0.6262],
         [-0.5349],
         [-0.5436],
  

# Model

In [45]:
class MultipleRegression(nn.Module):
    def __init__(self, num_features):
        super(MultipleRegression, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 16)
        self.layer_2 = nn.Linear(16, 32)
        self.layer_3 = nn.Linear(32, 16)
        self.layer_out = nn.Linear(16, 1)
        
        self.relu = nn.ReLU()

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.relu(self.layer_2(x))
        x = self.relu(self.layer_3(x))
        x = self.layer_out(x)

        return x

In [46]:
model = MultipleRegression(num_features=5)

In [47]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [51]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

MultipleRegression(
  (layer_1): Linear(in_features=5, out_features=16, bias=True)
  (layer_2): Linear(in_features=16, out_features=32, bias=True)
  (layer_3): Linear(in_features=32, out_features=16, bias=True)
  (layer_out): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)

In [52]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [53]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [54]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.651941  [   16/ 3460]
loss: 0.870161  [  176/ 3460]
loss: 0.642314  [  336/ 3460]
loss: 0.655393  [  496/ 3460]
loss: 0.538557  [  656/ 3460]
loss: 0.637014  [  816/ 3460]
loss: 0.674752  [  976/ 3460]
loss: 0.603464  [ 1136/ 3460]
loss: 0.520536  [ 1296/ 3460]
loss: 0.451280  [ 1456/ 3460]
loss: 0.972002  [ 1616/ 3460]
loss: 1.123011  [ 1776/ 3460]
loss: 0.816682  [ 1936/ 3460]
loss: 0.503058  [ 2096/ 3460]
loss: 0.607497  [ 2256/ 3460]
loss: 0.653267  [ 2416/ 3460]
loss: 0.729381  [ 2576/ 3460]
loss: 0.561396  [ 2736/ 3460]
loss: 0.591647  [ 2896/ 3460]
loss: 0.486574  [ 3056/ 3460]
loss: 0.538101  [ 3216/ 3460]
loss: 0.593084  [ 3376/ 3460]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.563323 

Epoch 2
-------------------------------
loss: 0.452288  [   16/ 3460]
loss: 0.414511  [  176/ 3460]
loss: 1.188594  [  336/ 3460]
loss: 0.480943  [  496/ 3460]
loss: 0.760850  [  656/ 3460]
loss: 0.309972  [  816/ 3460]
loss: 0.624749  [  976/ 3460]